## Descriptors Project

In [1]:
import numbers

In [2]:
class IntegerField:
    def __init__(self, min_, max_):
        self._min = min_
        self._max = max_
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError(f'{self.prop_name} must be an integer value.')
        if value < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min}.')
        if value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name, None)

In [3]:
class Person:
    age = IntegerField(0, 100)

In [4]:
p = Person()

In [5]:
p.age = 5

In [6]:
p.age

5

In [7]:
try:
    p.age = 200
except ValueError as ex:
    print(ex)

age must be <= 100


In [8]:
import unittest

In [9]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [10]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)
        
    def test_set_age_ok(self):
        p = self.Person()
        p.age = 0
        self.assertEqual(0, p.age)

In [11]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField.test_set_age_ok) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [12]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)
        
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        self.Person.age = IntegerField(5, 10)
        
        p = self.Person()
        p.age = 5
        self.assertEqual(5, p.age)

In [13]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField.test_set_age_ok) ... ERROR

ERROR: test_set_age_ok (__main__.TestIntegerField.test_set_age_ok)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\LuizCláudioSouza\AppData\Local\Temp\ipykernel_24972\1558937223.py", line 11, in test_set_age_ok
    p.age = 5
    ^^^^^
  File "C:\Users\LuizCláudioSouza\AppData\Local\Temp\ipykernel_24972\739770774.py", line 16, in __set__
    instance.__dict__[self.prop_name] = value
                      ^^^^^^^^^^^^^^
AttributeError: 'IntegerField' object has no attribute 'prop_name'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


In [14]:
class TestIntegerField(unittest.TestCase):
    class Person:
        age = IntegerField(0, 10)
        
    def create_person(self, min_, max_):
        self.Person.age = IntegerField(min_, max_)
        self.Person.age.__set_name__(Person, 'age')
        return self.Person()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_person(min_, max_)
        p.age = 5
        self.assertEqual(5, p.age)

In [15]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField.test_set_age_ok) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [16]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        p = self.create_test_class(min_, max_)
        
        p.age = 5
        self.assertEqual(5, p.age)

In [17]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField.test_set_age_ok) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [19]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
    
    def test_set_age_ok(self):
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_ + 1)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)

In [20]:
run_tests(TestIntegerField)

test_set_age_ok (__main__.TestIntegerField.test_set_age_ok) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [21]:
class TestIntegerField(unittest.TestCase):
    @staticmethod
    def create_test_class(min_, max_):
        obj = type('TestClass', (), {'age': IntegerField(min_, max_)})
        return obj()
    
    def test_set_age_ok(self):
        '''Tests that valid values can be assigned/retrieved'''
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        valid_values = range(min_, max_ + 1)
        
        for i, value in enumerate(valid_values):
            with self.subTest(test_number=i):
                obj.age = value
                self.assertEqual(value, obj.age)
                
    def test_set_age_invalid(self):
        '''Tests that invalid values raise ValueError exceptions'''
        min_ = 5
        max_ = 10
        obj = self.create_test_class(min_, max_)
        bad_values = list(range(min_ -5, min_))
        bad_values += list(range(max_ +1, max_ + 5))
        bad_values += [10.5, 1+0j, 'abc']
        
        for i, value in enumerate(bad_values):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    obj.age = value
                    
    def test_class_get(self):
        '''Tests that class attribute retrieval returns the descriptor instance'''
        obj = self.create_test_class(0, 0)
        obj_class = type(obj)
        self.assertIsInstance(obj_class.age, IntegerField)    

In [22]:
run_tests(TestIntegerField)

test_class_get (__main__.TestIntegerField.test_class_get)
Tests that class attribute retrieval returns the descriptor instance ... ok
test_set_age_invalid (__main__.TestIntegerField.test_set_age_invalid)
Tests that invalid values raise ValueError exceptions ... ok
test_set_age_ok (__main__.TestIntegerField.test_set_age_ok)
Tests that valid values can be assigned/retrieved ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK
